We are uploading all csv files of the Scarlet & Violet main and special sets, then merging all 13 sets into a master set.

Sets: SV01, SV02, SV03, SV: Scarlet & Violet 151, SV04, SV: Paldean Fates, SV05, SV06, SV: Shrouded Fable, SV07, SV08, SV: Prismatic Evolutions, SV09

In [2]:
import pandas as pd
sv01_df = pd.read_csv('SV01ScarletAndVioletBaseSetProductsAndPrices.csv')
sv02_df = pd.read_csv('SV02PaldeaEvolvedProductsAndPrices.csv')
sv03_df = pd.read_csv('SV03ObsidianFlamesProductsAndPrices.csv')
sv04_df = pd.read_csv('SV04ParadoxRiftProductsAndPrices.csv')
sv05_df = pd.read_csv('SV05TemporalForcesProductsAndPrices.csv')
sv06_df = pd.read_csv('SV06TwilightMasqueradeProductsAndPrices.csv')
sv07_df = pd.read_csv('SV07StellarCrownProductsAndPrices.csv')
sv08_df = pd.read_csv('SV08SurgingSparksProductsAndPrices.csv')
sv09_df = pd.read_csv('SV09JourneyTogetherProductsAndPrices.csv')
sv_151_df = pd.read_csv('SVScarletAndViolet151ProductsAndPrices.csv')
sv_pf_df = pd.read_csv('SVPaldeanFatesProductsAndPrices.csv')
sv_sf_df = pd.read_csv('SVShroudedFableProductsAndPrices.csv')
sv_pe_df = pd.read_csv('SVPrismaticEvolutionsProductsAndPrices.csv')

In [ ]:
master_df = pd.concat([sv01_df, sv02_df, sv03_df, sv04_df, sv05_df, sv06_df, sv07_df, sv08_df, sv09_df, sv_151_df, sv_pf_df, sv_sf_df, sv_pe_df])
master_df = master_df[['productId', 'name', 'groupId', 'marketPrice', 'extRarity']]

master_df = master_df[(master_df['extRarity'] == 'Special Illustration Rare') | (master_df['extRarity'] == 'Illustration Rare') | (master_df['extRarity'] == 'Hyper Rare')]

master_df = master_df.reset_index()
master_df.index +=1





master_df

,index,productId,name,groupId,marketPrice,extRarity
1,52,485259,Miraidon ex - 244/198,22873,24.49,Special Illustration Rare
2,62,486622,Dolliv - 200/198,22873,5.54,Illustration Rare
3,63,486623,Gardevoir ex - 245/198,22873,51.91,Special Illustration Rare
4,74,487060,Dondozo - 207/198,22873,11.47,Illustration Rare
5,75,487061,Miriam - 251/198,22873,28.78,Special Illustration Rare
...,...,...,...,...,...,...
437,304,610531,Iron Leaves ex,23821,10.25,Hyper Rare
438,305,610532,Teal Mask Ogerpon ex - 177/131,23821,17.80,Hyper Rare
439,306,610533,Walking Wake ex,23821,13.94,Hyper Rare
440,307,610534,Pikachu ex - 179/131,23821,78.57,Hyper Rare
